<a href="https://colab.research.google.com/github/senjoyee/udemy_langchain_python/blob/main/langchain_python_bootcamp_2_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-openai

In [2]:
!pip install pydantic

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate,AIMessagePromptTemplate
from google.colab import userdata

In [4]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
model = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
output_parser = CommaSeparatedListOutputParser()

In [6]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [7]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [8]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [9]:
chat_prompt.format_prompt(request="give me 5 benefits of eating eggs",
                   format_instructions = output_parser.get_format_instructions())

ChatPromptValue(messages=[HumanMessage(content='give me 5 benefits of eating eggs\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')])

In [10]:
model_request = chat_prompt.format_prompt(request="write a poem about animals",
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [11]:
model_request

[HumanMessage(content='write a poem about animals\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')]

In [25]:
result = model.invoke(model_request)

In [13]:
print(result.content)

Lions, tigers, bears,
Wolves, elephants, hares,
Dolphins, eagles, snakes,
Kangaroos, monkeys, drakes,
Animals big and small,
Wonderful creatures, one and all.


In [14]:
output_parser.parse(result.content)

['Lions',
 'tigers',
 'bears',
 'Wolves',
 'elephants',
 'hares',
 'Dolphins',
 'eagles',
 'snakes',
 'Kangaroos',
 'monkeys',
 'drakes',
 'Animals big and small',
 'Wonderful creatures',
 'one and all.']

In [15]:
from langchain_core.output_parsers import PydanticOutputParser

In [16]:
from pydantic import BaseModel, Field

In [17]:
class Scientist(BaseModel):

  name: str = Field(description="Name of a scientist")
  discoveries: list = Field(description="Python list of discoveries")

In [18]:
py_parser = PydanticOutputParser(pydantic_object=Scientist)

In [19]:
print(py_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [20]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")

In [21]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [22]:
request = chat_prompt.format_prompt(request="Tell me about a famous scientist",
                                    format_instructions=py_parser.get_format_instructions()).to_messages()

In [23]:
result = model(request, temperature=0)

In [24]:
py_parser.parse(result.content)

Scientist(name='Isaac Newton', discoveries=['Law of Universal Gravitation', 'Three Laws of Motion'])